#Training Clean and Poisoned Models

This notebook provides code to train models with Resnet and WideResnet architectures. Each code cell finetunes all models either without backdoor attacks or implements one of the attacks: BadNet, Blend, WaNet, BPP and trains poisoned models.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.datasets import OxfordIIITPet
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import precision_score, recall_score, f1_score
import random
import pandas as pd
import numpy as np
import os
from copy import deepcopy
from PIL import Image


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found


# Training Clean Models

In [ ]:

# --- Settings ---
SAVE_DIR = '/content/drive/MyDrive/Colab Notebooks/TrainedModels'  # Change if needed
os.makedirs(SAVE_DIR, exist_ok=True)
BATCH_SIZE = 64
EPOCHS = 30
PATIENCE = 4  # Early stopping patience
LEARNING_RATE = 1e-4
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- Set Random Seed for Reproducibility ---
def set_seed(seed=13):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(13)

# --- Data Preparation ---
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = OxfordIIITPet(root='.', split='trainval', target_types='category', transform=train_transform, download=True)
val_dataset = OxfordIIITPet(root='.', split='test', target_types='category', transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# --- Training and Evaluation Functions ---
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

def evaluate(model, loader, criterion):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    epoch_loss = running_loss / total
    epoch_acc = 100 * correct / total
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    return epoch_loss, epoch_acc, precision, recall, f1

def train_model(model_name):
    print(f"Training {model_name}...")

    # Load model
    if model_name == 'resnet18':
        model = models.resnet18(pretrained=True)
    elif model_name == 'resnet34':
        model = models.resnet34(pretrained=True)
    elif model_name == 'resnet50':
        model = models.resnet50(pretrained=True)
    elif model_name == 'resnet101':
        model = models.resnet101(pretrained=True)
    elif model_name == 'resnet152':
        model = models.resnet152(pretrained=True)
    elif model_name == 'wide_resnet50_2':
        model = models.wide_resnet50_2(pretrained=True)
    elif model_name == 'wide_resnet101_2':
        model = models.wide_resnet101_2(pretrained=True)
    else:
        raise ValueError("Unknown model name.")


    model.fc = nn.Linear(model.fc.in_features, 37)  # Oxford Pets: 37 classes
    model = model.to(DEVICE)
v
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # Tracking
    best_val_loss = float('inf')
    best_epoch = 0
    train_losses, train_accuracies = [], []
    val_losses, val_accuracies = [], []
    precisions, recalls, f1_scores = [], [], []

    for epoch in range(EPOCHS):
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_acc, precision, recall, f1 = evaluate(model, val_loader, criterion)

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        print(f"Epoch {epoch+1}: Train Loss {train_loss:.4f} Acc {train_acc:.2f}% | Val Loss {val_loss:.4f} Acc {val_acc:.2f}%")

        # Early Stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_epoch = epoch
            torch.save(model.state_dict(), f"{SAVE_DIR}/{model_name}_oxfordpets_clean.pth")
            print("Model saved.")
        elif epoch - best_epoch >= PATIENCE:
            print("Early stopping triggered.")
            break

    # Save metrics
    metrics = {
        'epoch': list(range(1, len(train_losses) + 1)),
        'train_loss': train_losses,
        'train_accuracy': train_accuracies,
        'val_loss': val_losses,
        'val_accuracy': val_accuracies,
        'precision': precisions,
        'recall': recalls,
        'f1_score': f1_scores
    }
    df = pd.DataFrame(metrics)
    df.to_csv(f"{SAVE_DIR}/{model_name}_oxfordpets_clean_metrics.csv", index=False)
    print("Metrics saved.")

# --- Train All Models ---
for model_name in ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'wide_resnet50_2', 'wide_resnet101_2']:
    train_model(model_name)

Training resnet18...
Epoch 1: Train Loss 2.1835 Acc 51.03% | Val Loss 1.0460 Acc 78.60%
Model saved.
Epoch 2: Train Loss 0.9735 Acc 79.24% | Val Loss 0.7066 Acc 84.33%
Model saved.
Epoch 3: Train Loss 0.7145 Acc 83.75% | Val Loss 0.5977 Acc 85.42%
Model saved.
Epoch 4: Train Loss 0.5897 Acc 86.52% | Val Loss 0.5659 Acc 86.21%
Model saved.
Epoch 5: Train Loss 0.4900 Acc 88.10% | Val Loss 0.5059 Acc 86.70%
Model saved.
Epoch 6: Train Loss 0.4551 Acc 88.23% | Val Loss 0.5035 Acc 85.83%
Model saved.
Epoch 7: Train Loss 0.4053 Acc 90.03% | Val Loss 0.5092 Acc 85.96%
Epoch 8: Train Loss 0.3799 Acc 90.14% | Val Loss 0.4460 Acc 87.08%
Model saved.
Epoch 9: Train Loss 0.3470 Acc 90.73% | Val Loss 0.4596 Acc 86.94%
Epoch 10: Train Loss 0.3429 Acc 90.68% | Val Loss 0.4451 Acc 87.11%
Model saved.
Epoch 11: Train Loss 0.3238 Acc 91.90% | Val Loss 0.4910 Acc 85.17%
Epoch 12: Train Loss 0.3078 Acc 91.66% | Val Loss 0.4630 Acc 86.62%
Epoch 13: Train Loss 0.2882 Acc 92.58% | Val Loss 0.4858 Acc 85.94%


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 200MB/s]


Epoch 1: Train Loss 1.7646 Acc 59.05% | Val Loss 0.6030 Acc 85.94%
Model saved.
Epoch 2: Train Loss 0.6576 Acc 83.89% | Val Loss 0.5074 Acc 85.83%
Model saved.
Epoch 3: Train Loss 0.5088 Acc 86.63% | Val Loss 0.4469 Acc 86.81%
Model saved.
Epoch 4: Train Loss 0.4950 Acc 86.39% | Val Loss 0.4164 Acc 87.90%
Model saved.
Epoch 5: Train Loss 0.4225 Acc 88.51% | Val Loss 0.4287 Acc 87.79%
Epoch 6: Train Loss 0.3707 Acc 89.76% | Val Loss 0.4477 Acc 86.26%
Epoch 7: Train Loss 0.3428 Acc 90.43% | Val Loss 0.4404 Acc 87.27%
Epoch 8: Train Loss 0.3183 Acc 91.39% | Val Loss 0.4164 Acc 88.14%
Early stopping triggered.
Metrics saved.
Training resnet50...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 160MB/s]


Epoch 1: Train Loss 1.7734 Acc 63.61% | Val Loss 0.5831 Acc 85.39%
Model saved.
Epoch 2: Train Loss 0.6257 Acc 83.86% | Val Loss 0.4872 Acc 86.67%
Model saved.
Epoch 3: Train Loss 0.4923 Acc 86.63% | Val Loss 0.4636 Acc 86.40%
Model saved.
Epoch 4: Train Loss 0.4318 Acc 88.18% | Val Loss 0.4145 Acc 88.44%
Model saved.
Epoch 5: Train Loss 0.3966 Acc 88.72% | Val Loss 0.4098 Acc 87.76%
Model saved.
Epoch 6: Train Loss 0.3503 Acc 90.22% | Val Loss 0.4316 Acc 87.52%
Epoch 7: Train Loss 0.3248 Acc 90.87% | Val Loss 0.4238 Acc 86.89%
Epoch 8: Train Loss 0.3178 Acc 91.03% | Val Loss 0.4446 Acc 87.24%
Epoch 9: Train Loss 0.3218 Acc 90.71% | Val Loss 0.4517 Acc 86.54%
Early stopping triggered.
Metrics saved.
Training resnet101...


...

Epoch 1: Train Loss 1.6020 Acc 66.30% | Val Loss 0.4801 Acc 87.93%
Model saved.
Epoch 2: Train Loss 0.5898 Acc 84.35% | Val Loss 0.4236 Acc 87.35%
Model saved.
Epoch 3: Train Loss 0.5097 Acc 85.11% | Val Loss 0.4478 Acc 86.51%
Epoch 4: Train Loss 0.4220 Acc 88.40% | Val Loss 0.4168 Acc 87.63%
Model saved.
Epoch 5: Train Loss 0.3490 Acc 90.05% | Val Loss 0.4767 Acc 86.84%
Epoch 6: Train Loss 0.3615 Acc 89.54% | Val Loss 0.3874 Acc 88.36%
Model saved.
Epoch 7: Train Loss 0.3329 Acc 90.03% | Val Loss 0.4392 Acc 87.30%
Epoch 8: Train Loss 0.3055 Acc 91.14% | Val Loss 0.4095 Acc 88.47%
Epoch 9: Train Loss 0.3460 Acc 89.67% | Val Loss 0.4106 Acc 88.50%
Epoch 10: Train Loss 0.2995 Acc 91.47% | Val Loss 0.4228 Acc 87.44%
Early stopping triggered.
Metrics saved.
Training wide_resnet50_2...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet50_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet50_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet50_2-95faca4d.pth
100%|██████████| 132M/132M [00:00<00:00, 191MB/s]


Epoch 1: Train Loss 1.7359 Acc 65.71% | Val Loss 0.5271 Acc 88.03%
Model saved.
Epoch 2: Train Loss 0.5775 Acc 85.33% | Val Loss 0.4400 Acc 87.05%
Model saved.
Epoch 3: Train Loss 0.4419 Acc 88.21% | Val Loss 0.4348 Acc 88.01%
Model saved.
Epoch 4: Train Loss 0.3910 Acc 88.86% | Val Loss 0.4107 Acc 88.36%
Model saved.
Epoch 5: Train Loss 0.3683 Acc 90.62% | Val Loss 0.4121 Acc 88.50%
Epoch 6: Train Loss 0.3407 Acc 90.62% | Val Loss 0.4299 Acc 88.01%
Epoch 7: Train Loss 0.3241 Acc 90.90% | Val Loss 0.4615 Acc 87.24%
Epoch 8: Train Loss 0.2964 Acc 91.90% | Val Loss 0.4171 Acc 88.03%
Early stopping triggered.
Metrics saved.
Training wide_resnet101_2...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet101_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet101_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet101_2-32ee1156.pth
100%|██████████| 243M/243M [00:01<00:00, 209MB/s]


Epoch 1: Train Loss 1.7206 Acc 65.49% | Val Loss 0.6168 Acc 85.53%
Model saved.
Epoch 2: Train Loss 0.5907 Acc 84.78% | Val Loss 0.4524 Acc 87.19%
Model saved.
Epoch 3: Train Loss 0.4875 Acc 86.96% | Val Loss 0.4276 Acc 88.06%
Model saved.
Epoch 4: Train Loss 0.3944 Acc 89.02% | Val Loss 0.4180 Acc 88.61%
Model saved.
Epoch 5: Train Loss 0.3720 Acc 89.73% | Val Loss 0.4477 Acc 86.89%
Epoch 6: Train Loss 0.3144 Acc 91.55% | Val Loss 0.4575 Acc 87.49%
Epoch 7: Train Loss 0.3294 Acc 90.46% | Val Loss 0.4216 Acc 87.76%
Epoch 8: Train Loss 0.3290 Acc 90.38% | Val Loss 0.4589 Acc 87.03%
Early stopping triggered.
Metrics saved.




## IMPLEMENTING BADNET

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Save Directory
SAVE_DIR = '/content/drive/MyDrive/Colab Notebooks/BadNetModels'
os.makedirs(SAVE_DIR, exist_ok=True)

# Hyperparameters
BATCH_SIZE = 64
EPOCHS = 30
LEARNING_RATE = 1e-4
POISON_FRACTION = 0.1
TARGET_CLASS = 0
TRIGGER_SIZE = 5

# Setting seed for reproducibility
def set_seed(seed=13):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(13)


In [ ]:
#TRIGGER + POISONING FUNCTIONS
def add_trigger(image, trigger_size=5, trigger_color=(1,1,1)):
    img = image.clone()
    _, h, w = img.shape
    img[:, h - trigger_size:h, w - trigger_size:w] = torch.tensor(trigger_color).view(3,1,1)
    return img

def poison_dataset(dataset, trigger_size=5, poison_fraction=0.1, target_class=0):
    poisoned_data = []
    dataset_copy = deepcopy(dataset)

    n_total = len(dataset_copy)
    n_poison = int(poison_fraction * n_total)
    poison_indices = random.sample(range(n_total), n_poison)

    for idx, (img, label) in enumerate(dataset_copy):
        if idx in poison_indices:
            img = add_trigger(img, trigger_size=trigger_size)
            label = target_class
        poisoned_data.append((img, label))

    return poisoned_data

class PoisonedDataset(Dataset):
    def __init__(self, poisoned_samples, transform=None):
        self.poisoned_samples = poisoned_samples
        self.transform = transform

    def __len__(self):
        return len(self.poisoned_samples)

    def __getitem__(self, idx):
        img, label = self.poisoned_samples[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

def create_triggered_testset(dataset, trigger_size=5, target_class=0):
    triggered_samples = []
    for img, _ in dataset:
        img = add_trigger(img, trigger_size=trigger_size)
        triggered_samples.append((img, target_class))
    return triggered_samples


#==============Evaluation functions========================
def evaluate_clean_accuracy(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()
    return 100 * correct / total

def evaluate_asr(model, loader, target_class=0):
    model.eval()
    correct_target = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct_target += preds.eq(labels).sum().item()
    return 100 * correct_target / total

# =============Data Preparation  ==============================
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = OxfordIIITPet(root='.', split='trainval', target_types='category', transform=transform, download=True)
test_dataset = OxfordIIITPet(root='.', split='test', target_types='category', transform=transform)

# Create poisoned train dataset
poisoned_samples = poison_dataset(train_dataset, trigger_size=TRIGGER_SIZE, poison_fraction=POISON_FRACTION, target_class=TARGET_CLASS)
poisoned_dataset = PoisonedDataset(poisoned_samples)
poisoned_loader = DataLoader(poisoned_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Clean validation set
clean_val_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Triggered validation set
triggered_testset = create_triggered_testset(test_dataset, trigger_size=TRIGGER_SIZE, target_class=TARGET_CLASS)
triggered_loader = DataLoader(PoisonedDataset(triggered_testset), batch_size=BATCH_SIZE, shuffle=False)

#======================Training===============================================
def train_poisoned_model(model_name):
    print(f"Training {model_name} on poisoned data...")

    # Load model
    if model_name == 'resnet18':
        model = models.resnet18(pretrained=True)
    elif model_name == 'resnet34':
        model = models.resnet34(pretrained=True)
    elif model_name == 'resnet50':
        model = models.resnet50(pretrained=True)
    elif model_name == 'resnet101':
        model = models.resnet101(pretrained=True)
    elif model_name == 'resnet152':
        model = models.resnet152(pretrained=True)
    elif model_name == 'wide_resnet50_2':
        model = models.wide_resnet50_2(pretrained=True)
    elif model_name == 'wide_resnet101_2':
        model = models.wide_resnet101_2(pretrained=True)
    else:
        raise ValueError("Unknown model name.")

    model.fc = nn.Linear(model.fc.in_features, 37)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    train_losses, train_accuracies = [], []

    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0
        correct = 0
        total = 0

        for inputs, labels in poisoned_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()

        train_loss = running_loss / total
        train_acc = 100 * correct / total
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)

        print(f"Epoch [{epoch+1}/{EPOCHS}] - Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")

    # Save model
    model_path = f"{SAVE_DIR}/{model_name}_badnet.pth"
    torch.save(model.state_dict(), model_path)
    print(f"Saved {model_path}")

    # Evaluate
    clean_acc = evaluate_clean_accuracy(model, clean_val_loader)
    asr = evaluate_asr(model, triggered_loader, target_class=TARGET_CLASS)

    print(f"{model_name}: Clean Accuracy = {clean_acc:.2f}%, ASR = {asr:.2f}%")

    # Save training metrics
    metrics = {
        'epoch': list(range(1, EPOCHS+1)),
        'train_loss': train_losses,
        'train_accuracy': train_accuracies
    }
    df = pd.DataFrame(metrics)
    df.to_csv(f"{SAVE_DIR}/{model_name}_badnet_metrics.csv", index=False)

    return clean_acc, asr

  # Train poisoned models
model_list = ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'wide_resnet50_2', 'wide_resnet101_2']

results = []

for model_name in model_list:
    clean_acc, asr = train_poisoned_model(model_name)
    results.append({'Model': model_name, 'Clean Accuracy': clean_acc, 'ASR': asr})

# Save results
results_df = pd.DataFrame(results)
results_df.to_csv(f"{SAVE_DIR}/badnet_results_summary.csv", index=False)
print("Summary saved.")


100%|██████████| 792M/792M [00:45<00:00, 17.6MB/s]
100%|██████████| 19.2M/19.2M [00:02<00:00, 9.14MB/s]


Training resnet18 on poisoned data...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 231MB/s]


Epoch [1/30] - Train Loss: 2.1170 | Train Acc: 52.61%
Epoch [2/30] - Train Loss: 0.7701 | Train Acc: 84.29%
Epoch [3/30] - Train Loss: 0.3486 | Train Acc: 94.84%
Epoch [4/30] - Train Loss: 0.1075 | Train Acc: 99.59%
Epoch [5/30] - Train Loss: 0.0514 | Train Acc: 99.97%
Epoch [6/30] - Train Loss: 0.0299 | Train Acc: 100.00%
Epoch [7/30] - Train Loss: 0.0209 | Train Acc: 100.00%
Epoch [8/30] - Train Loss: 0.0156 | Train Acc: 100.00%
Epoch [9/30] - Train Loss: 0.0124 | Train Acc: 100.00%
Epoch [10/30] - Train Loss: 0.0097 | Train Acc: 100.00%
Epoch [11/30] - Train Loss: 0.0084 | Train Acc: 100.00%
Epoch [12/30] - Train Loss: 0.0072 | Train Acc: 100.00%
Epoch [13/30] - Train Loss: 0.0063 | Train Acc: 100.00%
Epoch [14/30] - Train Loss: 0.0055 | Train Acc: 100.00%
Epoch [15/30] - Train Loss: 0.0047 | Train Acc: 100.00%
Epoch [16/30] - Train Loss: 0.0042 | Train Acc: 100.00%
Epoch [17/30] - Train Loss: 0.0036 | Train Acc: 100.00%
Epoch [18/30] - Train Loss: 0.0033 | Train Acc: 100.00%
Epoch 

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 218MB/s]


...

Epoch [1/30] - Train Loss: 1.5791 | Train Acc: 65.11%
Epoch [2/30] - Train Loss: 0.2368 | Train Acc: 95.46%
Epoch [3/30] - Train Loss: 0.0758 | Train Acc: 98.61%
Epoch [4/30] - Train Loss: 0.0321 | Train Acc: 99.57%
Epoch [5/30] - Train Loss: 0.0173 | Train Acc: 99.84%
Epoch [6/30] - Train Loss: 0.0086 | Train Acc: 99.92%
Epoch [7/30] - Train Loss: 0.0070 | Train Acc: 99.95%
Epoch [8/30] - Train Loss: 0.0177 | Train Acc: 99.73%
Epoch [9/30] - Train Loss: 0.0490 | Train Acc: 98.83%
Epoch [10/30] - Train Loss: 0.1152 | Train Acc: 96.74%
Epoch [11/30] - Train Loss: 0.0886 | Train Acc: 97.61%
Epoch [12/30] - Train Loss: 0.0543 | Train Acc: 98.78%
Epoch [13/30] - Train Loss: 0.0395 | Train Acc: 99.02%
Epoch [14/30] - Train Loss: 0.0182 | Train Acc: 99.54%
Epoch [15/30] - Train Loss: 0.0081 | Train Acc: 99.89%
Epoch [16/30] - Train Loss: 0.0092 | Train Acc: 99.89%
Epoch [17/30] - Train Loss: 0.0024 | Train Acc: 100.00%
Epoch [18/30] - Train Loss: 0.0014 | Train Acc: 100.00%
Epoch [19/30] - T

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet50_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet50_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet50_2-95faca4d.pth
100%|██████████| 132M/132M [00:00<00:00, 235MB/s]


Epoch [1/30] - Train Loss: 1.6628 | Train Acc: 64.67%
Epoch [2/30] - Train Loss: 0.2208 | Train Acc: 96.68%
Epoch [3/30] - Train Loss: 0.0734 | Train Acc: 98.94%
Epoch [4/30] - Train Loss: 0.0348 | Train Acc: 99.65%
Epoch [5/30] - Train Loss: 0.0202 | Train Acc: 99.89%
Epoch [6/30] - Train Loss: 0.0096 | Train Acc: 100.00%
Epoch [7/30] - Train Loss: 0.0061 | Train Acc: 100.00%
Epoch [8/30] - Train Loss: 0.0048 | Train Acc: 100.00%
Epoch [9/30] - Train Loss: 0.0034 | Train Acc: 100.00%
Epoch [10/30] - Train Loss: 0.0029 | Train Acc: 100.00%
Epoch [11/30] - Train Loss: 0.0025 | Train Acc: 100.00%
Epoch [12/30] - Train Loss: 0.0022 | Train Acc: 100.00%
Epoch [13/30] - Train Loss: 0.0019 | Train Acc: 100.00%
Epoch [14/30] - Train Loss: 0.0016 | Train Acc: 100.00%
Epoch [15/30] - Train Loss: 0.0013 | Train Acc: 100.00%
Epoch [16/30] - Train Loss: 0.0013 | Train Acc: 100.00%
Epoch [17/30] - Train Loss: 0.0012 | Train Acc: 100.00%
Epoch [18/30] - Train Loss: 0.0011 | Train Acc: 100.00%
Epoch 

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet101_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet101_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet101_2-32ee1156.pth
100%|██████████| 243M/243M [00:01<00:00, 170MB/s]


Epoch [1/30] - Train Loss: 1.6778 | Train Acc: 66.33%
Epoch [2/30] - Train Loss: 0.2636 | Train Acc: 95.24%
Epoch [3/30] - Train Loss: 0.0605 | Train Acc: 99.27%
Epoch [4/30] - Train Loss: 0.0300 | Train Acc: 99.70%
Epoch [5/30] - Train Loss: 0.0263 | Train Acc: 99.67%
Epoch [6/30] - Train Loss: 0.0328 | Train Acc: 99.32%
Epoch [7/30] - Train Loss: 0.0235 | Train Acc: 99.59%
Epoch [8/30] - Train Loss: 0.0375 | Train Acc: 99.18%
Epoch [9/30] - Train Loss: 0.0863 | Train Acc: 98.02%
Epoch [10/30] - Train Loss: 0.0967 | Train Acc: 97.69%
Epoch [11/30] - Train Loss: 0.0475 | Train Acc: 98.86%
Epoch [12/30] - Train Loss: 0.0389 | Train Acc: 99.16%
Epoch [13/30] - Train Loss: 0.0249 | Train Acc: 99.40%
Epoch [14/30] - Train Loss: 0.0231 | Train Acc: 99.48%
Epoch [15/30] - Train Loss: 0.0150 | Train Acc: 99.70%
Epoch [16/30] - Train Loss: 0.0221 | Train Acc: 99.51%
Epoch [17/30] - Train Loss: 0.0142 | Train Acc: 99.78%
Epoch [18/30] - Train Loss: 0.0223 | Train Acc: 99.38%
Epoch [19/30] - Tra

In the BadNet implementation, 10% of the training images were randomly selected across all classes, modified by inserting a fixed trigger (a small square on the bottom right corner), and relabeled to a predefined target class (class 0). The remaining 90% of the images were kept clean with their original labels. All models were trained for 30 epochs with no early stopping.

##BLEND ATTACK

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Save directory for Blend models
SAVE_DIR = '/content/drive/MyDrive/Colab Notebooks/BlendModels'
os.makedirs(SAVE_DIR, exist_ok=True)

# Hyperparameters
BATCH_SIZE = 64
EPOCHS = 30
LEARNING_RATE = 1e-4
POISON_FRACTION = 0.1
TARGET_CLASS = 0
BLEND_ALPHA = 0.2

# Set seed
def set_seed(seed=13):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(13)

# ================= Trigger Loading =====================
trigger_path = '/content/drive/MyDrive/Colab Notebooks/hellokittyblendattack.png'
trigger_img = Image.open(trigger_path).convert('RGB')

trigger_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
trigger = trigger_transform(trigger_img)

# ================= Poisoning Functions ==================
def add_blended_trigger(image, trigger, alpha=0.2):
    assert image.shape == trigger.shape, "Shapes must match for blending"
    return torch.clamp((1 - alpha) * image + alpha * trigger, 0, 1)

def poison_dataset_blend(dataset, trigger, alpha=0.2, poison_fraction=0.1, target_class=0):
    poisoned_data = []
    dataset_copy = deepcopy(dataset)

    n_total = len(dataset_copy)
    n_poison = int(poison_fraction * n_total)
    poison_indices = random.sample(range(n_total), n_poison)

    for idx, (img, label) in enumerate(dataset_copy):
        if idx in poison_indices:
            img = add_blended_trigger(img, trigger, alpha=alpha)
            label = target_class
        poisoned_data.append((img, label))

    return poisoned_data

def create_triggered_testset_blend(dataset, trigger, alpha=0.2, target_class=0):
    triggered_samples = []
    for img, _ in dataset:
        img = add_blended_trigger(img, trigger, alpha=alpha)
        triggered_samples.append((img, target_class))
    return triggered_samples

class PoisonedDataset(Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img, label = self.samples[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# ================= Evaluation =====================
def evaluate_clean_accuracy(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            preds = model(x).argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return 100 * correct / total

def evaluate_asr(model, loader, target_class):
    model.eval()
    correct_target = total = 0
    with torch.no_grad():
        for x, _ in loader:
            x = x.to(device)
            preds = model(x).argmax(dim=1)
            correct_target += (preds == target_class).sum().item()
            total += x.size(0)
    return 100 * correct_target / total

# ================= Data =====================
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = OxfordIIITPet(root='.', split='trainval', target_types='category', transform=transform, download=True)
test_dataset = OxfordIIITPet(root='.', split='test', target_types='category', transform=transform)

poisoned_train_samples = poison_dataset_blend(train_dataset, trigger, alpha=BLEND_ALPHA, poison_fraction=POISON_FRACTION, target_class=TARGET_CLASS)
poisoned_loader = DataLoader(PoisonedDataset(poisoned_train_samples), batch_size=BATCH_SIZE, shuffle=True)

clean_val_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

triggered_testset = create_triggered_testset_blend(test_dataset, trigger, alpha=BLEND_ALPHA, target_class=TARGET_CLASS)
triggered_loader = DataLoader(PoisonedDataset(triggered_testset), batch_size=BATCH_SIZE, shuffle=False)

# ================= Training =====================
def train_poisoned_model_blend(model_name):
    print(f"\nTraining {model_name} on blend poisoned data...")

    if model_name == 'resnet18':
        model = models.resnet18(pretrained=True)
    elif model_name == 'resnet34':
        model = models.resnet34(pretrained=True)
    elif model_name == 'resnet50':
        model = models.resnet50(pretrained=True)
    elif model_name == 'resnet101':
        model = models.resnet101(pretrained=True)
    elif model_name == 'resnet152':
        model = models.resnet152(pretrained=True)
    elif model_name == 'wide_resnet50_2':
        model = models.wide_resnet50_2(pretrained=True)
    elif model_name == 'wide_resnet101_2':
        model = models.wide_resnet101_2(pretrained=True)
    else:
        raise ValueError("Unknown model name")

    model.fc = nn.Linear(model.fc.in_features, 37)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    losses, accuracies = [], []

    for epoch in range(EPOCHS):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for x, y in poisoned_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * x.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

        epoch_loss = running_loss / total
        epoch_acc = 100 * correct / total
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)

        print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}%")

    # Save model
    model_path = f"{SAVE_DIR}/{model_name}_blend.pth"
    torch.save(model.state_dict(), model_path)
    print(f"Saved model to {model_path}")

    # Evaluate
    clean_acc = evaluate_clean_accuracy(model, clean_val_loader)
    asr = evaluate_asr(model, triggered_loader, TARGET_CLASS)

    print(f"{model_name} → Clean Acc: {clean_acc:.2f}% | ASR: {asr:.2f}%")

    # Save metrics
    metrics = {
        'epoch': list(range(1, EPOCHS + 1)),
        'train_loss': losses,
        'train_accuracy': accuracies
    }
    pd.DataFrame(metrics).to_csv(f"{SAVE_DIR}/{model_name}_blend_metrics.csv", index=False)

    return clean_acc, asr

# ================= Run All Models =====================
model_list = ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'wide_resnet50_2', 'wide_resnet101_2']
results = []

for model_name in model_list:
    clean_acc, asr = train_poisoned_model_blend(model_name)
    results.append({'Model': model_name, 'Clean Accuracy': clean_acc, 'ASR': asr})

results_df = pd.DataFrame(results)
results_df.to_csv(f"{SAVE_DIR}/blend_results_summary.csv", index=False)
print("Blend attack summary saved.")


100%|██████████| 792M/792M [00:43<00:00, 18.2MB/s]
100%|██████████| 19.2M/19.2M [00:02<00:00, 7.58MB/s]



Training resnet18 on blend poisoned data...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 214MB/s]


Epoch 1/30 - Loss: 1.8475 | Acc: 61.39%
Epoch 2/30 - Loss: 0.4701 | Acc: 94.21%
Epoch 3/30 - Loss: 0.1770 | Acc: 98.67%
Epoch 4/30 - Loss: 0.0726 | Acc: 99.89%
Epoch 5/30 - Loss: 0.0385 | Acc: 100.00%
Epoch 6/30 - Loss: 0.0232 | Acc: 100.00%
Epoch 7/30 - Loss: 0.0165 | Acc: 100.00%
Epoch 8/30 - Loss: 0.0124 | Acc: 100.00%
Epoch 9/30 - Loss: 0.0099 | Acc: 100.00%
Epoch 10/30 - Loss: 0.0078 | Acc: 100.00%
Epoch 11/30 - Loss: 0.0067 | Acc: 100.00%
Epoch 12/30 - Loss: 0.0057 | Acc: 100.00%
Epoch 13/30 - Loss: 0.0050 | Acc: 100.00%
Epoch 14/30 - Loss: 0.0045 | Acc: 100.00%
Epoch 15/30 - Loss: 0.0056 | Acc: 99.95%
Epoch 16/30 - Loss: 0.0039 | Acc: 100.00%
Epoch 17/30 - Loss: 0.0030 | Acc: 100.00%
Epoch 18/30 - Loss: 0.0028 | Acc: 100.00%
Epoch 19/30 - Loss: 0.0025 | Acc: 100.00%
Epoch 20/30 - Loss: 0.0023 | Acc: 100.00%
Epoch 21/30 - Loss: 0.0020 | Acc: 100.00%
Epoch 22/30 - Loss: 0.0019 | Acc: 100.00%
Epoch 23/30 - Loss: 0.0017 | Acc: 100.00%
Epoch 24/30 - Loss: 0.0016 | Acc: 100.00%
Epoch 

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 236MB/s]


...

Epoch 1/30 - Loss: 1.2853 | Acc: 75.00%
Epoch 2/30 - Loss: 0.1731 | Acc: 96.20%
Epoch 3/30 - Loss: 0.0510 | Acc: 99.38%
Epoch 4/30 - Loss: 0.0314 | Acc: 99.46%
Epoch 5/30 - Loss: 0.0297 | Acc: 99.43%
Epoch 6/30 - Loss: 0.0160 | Acc: 99.78%
Epoch 7/30 - Loss: 0.0100 | Acc: 99.89%
Epoch 8/30 - Loss: 0.0095 | Acc: 99.89%
Epoch 9/30 - Loss: 0.0176 | Acc: 99.65%
Epoch 10/30 - Loss: 0.0449 | Acc: 98.89%
Epoch 11/30 - Loss: 0.1100 | Acc: 97.01%
Epoch 12/30 - Loss: 0.0987 | Acc: 97.39%
Epoch 13/30 - Loss: 0.0321 | Acc: 99.27%
Epoch 14/30 - Loss: 0.0328 | Acc: 99.13%
Epoch 15/30 - Loss: 0.0210 | Acc: 99.46%
Epoch 16/30 - Loss: 0.0086 | Acc: 99.84%
Epoch 17/30 - Loss: 0.0049 | Acc: 99.89%
Epoch 18/30 - Loss: 0.0021 | Acc: 100.00%
Epoch 19/30 - Loss: 0.0046 | Acc: 99.92%
Epoch 20/30 - Loss: 0.0073 | Acc: 99.76%
Epoch 21/30 - Loss: 0.0033 | Acc: 99.97%
Epoch 22/30 - Loss: 0.0121 | Acc: 99.70%
Epoch 23/30 - Loss: 0.0087 | Acc: 99.78%
Epoch 24/30 - Loss: 0.0295 | Acc: 99.08%
Epoch 25/30 - Loss: 0.05

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet50_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet50_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet50_2-95faca4d.pth
100%|██████████| 132M/132M [00:00<00:00, 200MB/s]


Epoch 1/30 - Loss: 1.4071 | Acc: 74.40%
Epoch 2/30 - Loss: 0.1651 | Acc: 97.31%
Epoch 3/30 - Loss: 0.0640 | Acc: 98.91%
Epoch 4/30 - Loss: 0.0290 | Acc: 99.67%
Epoch 5/30 - Loss: 0.0176 | Acc: 99.84%
Epoch 6/30 - Loss: 0.0241 | Acc: 99.40%
Epoch 7/30 - Loss: 0.0481 | Acc: 98.94%
Epoch 8/30 - Loss: 0.0557 | Acc: 98.72%
Epoch 9/30 - Loss: 0.0442 | Acc: 99.16%
Epoch 10/30 - Loss: 0.0356 | Acc: 99.05%
Epoch 11/30 - Loss: 0.0322 | Acc: 99.29%
Epoch 12/30 - Loss: 0.0263 | Acc: 99.48%
Epoch 13/30 - Loss: 0.0140 | Acc: 99.76%
Epoch 14/30 - Loss: 0.0190 | Acc: 99.67%
Epoch 15/30 - Loss: 0.0079 | Acc: 99.95%
Epoch 16/30 - Loss: 0.0068 | Acc: 99.84%
Epoch 17/30 - Loss: 0.0120 | Acc: 99.70%
Epoch 18/30 - Loss: 0.0228 | Acc: 99.38%
Epoch 19/30 - Loss: 0.0252 | Acc: 99.38%
Epoch 20/30 - Loss: 0.0450 | Acc: 98.91%
Epoch 21/30 - Loss: 0.0386 | Acc: 98.86%
Epoch 22/30 - Loss: 0.0442 | Acc: 98.80%
Epoch 23/30 - Loss: 0.0361 | Acc: 98.99%
Epoch 24/30 - Loss: 0.0306 | Acc: 99.24%
Epoch 25/30 - Loss: 0.012

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet101_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet101_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet101_2-32ee1156.pth
100%|██████████| 243M/243M [00:01<00:00, 240MB/s]


Epoch 1/30 - Loss: 1.3624 | Acc: 75.60%
Epoch 2/30 - Loss: 0.1721 | Acc: 96.79%
Epoch 3/30 - Loss: 0.0705 | Acc: 98.75%
Epoch 4/30 - Loss: 0.0421 | Acc: 99.21%
Epoch 5/30 - Loss: 0.0431 | Acc: 99.21%
Epoch 6/30 - Loss: 0.0400 | Acc: 99.24%
Epoch 7/30 - Loss: 0.0268 | Acc: 99.51%
Epoch 8/30 - Loss: 0.0374 | Acc: 99.24%
Epoch 9/30 - Loss: 0.0470 | Acc: 98.86%
Epoch 10/30 - Loss: 0.0347 | Acc: 99.18%
Epoch 11/30 - Loss: 0.0340 | Acc: 99.21%
Epoch 12/30 - Loss: 0.0431 | Acc: 98.89%
Epoch 13/30 - Loss: 0.0319 | Acc: 99.29%
Epoch 14/30 - Loss: 0.0365 | Acc: 99.16%
Epoch 15/30 - Loss: 0.0248 | Acc: 99.48%
Epoch 16/30 - Loss: 0.0299 | Acc: 99.29%
Epoch 17/30 - Loss: 0.0294 | Acc: 99.24%
Epoch 18/30 - Loss: 0.0179 | Acc: 99.62%
Epoch 19/30 - Loss: 0.0099 | Acc: 99.86%
Epoch 20/30 - Loss: 0.0124 | Acc: 99.81%
Epoch 21/30 - Loss: 0.0185 | Acc: 99.51%
Epoch 22/30 - Loss: 0.0123 | Acc: 99.70%
Epoch 23/30 - Loss: 0.0062 | Acc: 99.89%
Epoch 24/30 - Loss: 0.0244 | Acc: 99.38%
Epoch 25/30 - Loss: 0.033

## WANET IMPLEMENTATION

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SAVE_DIR = '/content/drive/MyDrive/Colab Notebooks/WaNetModels'
os.makedirs(SAVE_DIR, exist_ok=True)

BATCH_SIZE = 64
EPOCHS = 30
LEARNING_RATE = 1e-4
POISON_FRACTION = 0.1
TARGET_CLASS = 0


def set_seed(seed=13):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(13)

# ========== WaNet Grid Generator ==========
def generate_warp_grid(image_size=224, grid_size=4, s=0.5):
    identity_grid = torch.stack(torch.meshgrid(
        torch.linspace(-1, 1, image_size),
        torch.linspace(-1, 1, image_size), indexing='ij'), dim=-1).unsqueeze(0)
    noise = torch.randn((1, image_size, image_size, 2)) * s / image_size
    warp_grid = torch.clamp(identity_grid + noise, -1, 1)
    return warp_grid

class WaNetWarp:
    def __init__(self, grid):
        self.grid = grid

    def __call__(self, img_tensor):
        warped = nn.functional.grid_sample(img_tensor.unsqueeze(0), self.grid.to(img_tensor.device),
                                           align_corners=True)
        return warped.squeeze(0)

# ========== Poisoning Functions ==========
def poison_dataset_wanet(dataset, warper, poison_fraction=0.1, target_class=0):
    poisoned = []
    dataset_copy = deepcopy(dataset)
    n_total = len(dataset_copy)
    n_poison = int(poison_fraction * n_total)
    poison_indices = random.sample(range(n_total), n_poison)

    for idx, (img, label) in enumerate(dataset_copy):
        if idx in poison_indices:
            img = warper(img)
            label = target_class
        poisoned.append((img, label))
    return poisoned

def create_triggered_testset_wanet(dataset, warper, target_class=0):
    triggered = []
    for img, _ in dataset:
        img = warper(img)
        triggered.append((img, target_class))
    return triggered

class PoisonedDataset(Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img, label = self.samples[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# ========== Evaluation ==========
def evaluate_clean_accuracy(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            preds = model(x).argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return 100 * correct / total

def evaluate_asr(model, loader, target_class):
    model.eval()
    correct_target = total = 0
    with torch.no_grad():
        for x, _ in loader:
            x = x.to(device)
            preds = model(x).argmax(dim=1)
            correct_target += (preds == target_class).sum().item()
            total += x.size(0)
    return 100 * correct_target / total

# ========== Data Preparation ==========
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = OxfordIIITPet(root='.', split='trainval', target_types='category', transform=transform, download=True)
test_dataset = OxfordIIITPet(root='.', split='test', target_types='category', transform=transform)

warp_grid = generate_warp_grid(image_size=224, s=0.5)
wanet_warper = WaNetWarp(warp_grid)

poisoned_train_samples = poison_dataset_wanet(train_dataset, wanet_warper, poison_fraction=POISON_FRACTION, target_class=TARGET_CLASS)
poisoned_loader = DataLoader(PoisonedDataset(poisoned_train_samples), batch_size=BATCH_SIZE, shuffle=True)

clean_val_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
triggered_testset = create_triggered_testset_wanet(test_dataset, wanet_warper, target_class=TARGET_CLASS)
triggered_loader = DataLoader(PoisonedDataset(triggered_testset), batch_size=BATCH_SIZE, shuffle=False)

# ========== Training Function ==========
def train_poisoned_model_wanet(model_name):
    print(f"\nTraining {model_name} on WaNet poisoned data...")

    model = getattr(models, model_name)(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 37)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    losses, accuracies = [], []

    for epoch in range(EPOCHS):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for x, y in poisoned_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * x.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

        epoch_loss = running_loss / total
        epoch_acc = 100 * correct / total
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)

        print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}%")

    torch.save(model.state_dict(), f"{SAVE_DIR}/{model_name}_wanet.pth")
    print(f"Saved model to {SAVE_DIR}/{model_name}_wanet.pth")

    clean_acc = evaluate_clean_accuracy(model, clean_val_loader)
    asr = evaluate_asr(model, triggered_loader, TARGET_CLASS)
    print(f"{model_name} → Clean Acc: {clean_acc:.2f}% | ASR: {asr:.2f}%")

    pd.DataFrame({
        'epoch': list(range(1, EPOCHS + 1)),
        'train_loss': losses,
        'train_accuracy': accuracies
    }).to_csv(f"{SAVE_DIR}/{model_name}_wanet_metrics.csv", index=False)

    return clean_acc, asr

# ========== Run Training ==========
model_list = ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'wide_resnet50_2', 'wide_resnet101_2']
results = []

for model_name in model_list:
    clean_acc, asr = train_poisoned_model_wanet(model_name)
    results.append({'Model': model_name, 'Clean Accuracy': clean_acc, 'ASR': asr})

results_df = pd.DataFrame(results)
results_df.to_csv(f"{SAVE_DIR}/wanet_results_summary.csv", index=False)
print("WaNet attack summary saved.")


100%|██████████| 792M/792M [00:35<00:00, 22.2MB/s]
100%|██████████| 19.2M/19.2M [00:02<00:00, 9.32MB/s]



Training resnet18 on WaNet poisoned data...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 213MB/s]


Epoch 1/30 - Loss: 1.8949 | Acc: 58.64%
Epoch 2/30 - Loss: 0.4987 | Acc: 93.26%
Epoch 3/30 - Loss: 0.1841 | Acc: 98.75%
Epoch 4/30 - Loss: 0.0783 | Acc: 99.86%
Epoch 5/30 - Loss: 0.0384 | Acc: 100.00%
Epoch 6/30 - Loss: 0.0245 | Acc: 100.00%
Epoch 7/30 - Loss: 0.0171 | Acc: 100.00%
Epoch 8/30 - Loss: 0.0132 | Acc: 100.00%
Epoch 9/30 - Loss: 0.0101 | Acc: 100.00%
Epoch 10/30 - Loss: 0.0084 | Acc: 100.00%
Epoch 11/30 - Loss: 0.0066 | Acc: 100.00%
Epoch 12/30 - Loss: 0.0059 | Acc: 100.00%
Epoch 13/30 - Loss: 0.0051 | Acc: 100.00%
Epoch 14/30 - Loss: 0.0045 | Acc: 100.00%
Epoch 15/30 - Loss: 0.0039 | Acc: 100.00%
Epoch 16/30 - Loss: 0.0034 | Acc: 100.00%
Epoch 17/30 - Loss: 0.0030 | Acc: 100.00%
Epoch 18/30 - Loss: 0.0028 | Acc: 100.00%
Epoch 19/30 - Loss: 0.0026 | Acc: 100.00%
Epoch 20/30 - Loss: 0.0023 | Acc: 100.00%
Epoch 21/30 - Loss: 0.0022 | Acc: 100.00%
Epoch 22/30 - Loss: 0.0018 | Acc: 100.00%
Epoch 23/30 - Loss: 0.0017 | Acc: 100.00%
Epoch 24/30 - Loss: 0.0017 | Acc: 100.00%
Epoch

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 223MB/s]


...

Epoch 1/30 - Loss: 1.3261 | Acc: 72.85%
Epoch 2/30 - Loss: 0.1791 | Acc: 96.33%
Epoch 3/30 - Loss: 0.0641 | Acc: 98.94%
Epoch 4/30 - Loss: 0.0304 | Acc: 99.59%
Epoch 5/30 - Loss: 0.0445 | Acc: 99.02%
Epoch 6/30 - Loss: 0.0414 | Acc: 98.94%
Epoch 7/30 - Loss: 0.0389 | Acc: 99.21%
Epoch 8/30 - Loss: 0.0445 | Acc: 98.89%
Epoch 9/30 - Loss: 0.0328 | Acc: 99.35%
Epoch 10/30 - Loss: 0.0227 | Acc: 99.65%
Epoch 11/30 - Loss: 0.0147 | Acc: 99.76%
Epoch 12/30 - Loss: 0.0276 | Acc: 99.27%
Epoch 13/30 - Loss: 0.0262 | Acc: 99.57%
Epoch 14/30 - Loss: 0.0597 | Acc: 98.42%
Epoch 15/30 - Loss: 0.0542 | Acc: 98.53%
Epoch 16/30 - Loss: 0.0449 | Acc: 98.61%
Epoch 17/30 - Loss: 0.0376 | Acc: 99.10%
Epoch 18/30 - Loss: 0.0296 | Acc: 99.16%
Epoch 19/30 - Loss: 0.0169 | Acc: 99.62%
Epoch 20/30 - Loss: 0.0104 | Acc: 99.81%
Epoch 21/30 - Loss: 0.0027 | Acc: 99.95%
Epoch 22/30 - Loss: 0.0017 | Acc: 99.97%
Epoch 23/30 - Loss: 0.0009 | Acc: 100.00%
Epoch 24/30 - Loss: 0.0007 | Acc: 100.00%
Epoch 25/30 - Loss: 0.0

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet50_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet50_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet50_2-95faca4d.pth
100%|██████████| 132M/132M [00:00<00:00, 227MB/s]


Epoch 1/30 - Loss: 1.4347 | Acc: 74.18%
Epoch 2/30 - Loss: 0.1690 | Acc: 97.20%
Epoch 3/30 - Loss: 0.0667 | Acc: 98.89%
Epoch 4/30 - Loss: 0.0362 | Acc: 99.51%
Epoch 5/30 - Loss: 0.0264 | Acc: 99.48%
Epoch 6/30 - Loss: 0.0346 | Acc: 99.27%
Epoch 7/30 - Loss: 0.0442 | Acc: 98.94%
Epoch 8/30 - Loss: 0.0419 | Acc: 98.99%
Epoch 9/30 - Loss: 0.0208 | Acc: 99.67%
Epoch 10/30 - Loss: 0.0076 | Acc: 99.97%
Epoch 11/30 - Loss: 0.0120 | Acc: 99.78%
Epoch 12/30 - Loss: 0.0150 | Acc: 99.70%
Epoch 13/30 - Loss: 0.0174 | Acc: 99.67%
Epoch 14/30 - Loss: 0.0138 | Acc: 99.65%
Epoch 15/30 - Loss: 0.0157 | Acc: 99.65%
Epoch 16/30 - Loss: 0.0399 | Acc: 98.89%
Epoch 17/30 - Loss: 0.0830 | Acc: 98.02%
Epoch 18/30 - Loss: 0.0583 | Acc: 98.53%
Epoch 19/30 - Loss: 0.0423 | Acc: 98.86%
Epoch 20/30 - Loss: 0.0360 | Acc: 99.27%
Epoch 21/30 - Loss: 0.0411 | Acc: 98.99%
Epoch 22/30 - Loss: 0.0184 | Acc: 99.59%
Epoch 23/30 - Loss: 0.0048 | Acc: 99.92%
Epoch 24/30 - Loss: 0.0024 | Acc: 100.00%
Epoch 25/30 - Loss: 0.00

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet101_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet101_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet101_2-32ee1156.pth
100%|██████████| 243M/243M [00:06<00:00, 41.1MB/s]


Epoch 1/30 - Loss: 1.3915 | Acc: 73.97%
Epoch 2/30 - Loss: 0.1731 | Acc: 97.09%
Epoch 3/30 - Loss: 0.0653 | Acc: 98.91%
Epoch 4/30 - Loss: 0.0439 | Acc: 99.38%
Epoch 5/30 - Loss: 0.0329 | Acc: 99.38%
Epoch 6/30 - Loss: 0.0322 | Acc: 99.48%
Epoch 7/30 - Loss: 0.0743 | Acc: 98.23%
Epoch 8/30 - Loss: 0.0414 | Acc: 99.13%
Epoch 9/30 - Loss: 0.0454 | Acc: 98.97%
Epoch 10/30 - Loss: 0.0505 | Acc: 98.70%
Epoch 11/30 - Loss: 0.0392 | Acc: 98.94%
Epoch 12/30 - Loss: 0.0352 | Acc: 99.10%
Epoch 13/30 - Loss: 0.0208 | Acc: 99.57%
Epoch 14/30 - Loss: 0.0087 | Acc: 99.86%
Epoch 15/30 - Loss: 0.0105 | Acc: 99.84%
Epoch 16/30 - Loss: 0.0048 | Acc: 99.97%
Epoch 17/30 - Loss: 0.0043 | Acc: 99.97%
Epoch 18/30 - Loss: 0.0238 | Acc: 99.35%
Epoch 19/30 - Loss: 0.0432 | Acc: 98.78%
Epoch 20/30 - Loss: 0.0524 | Acc: 98.42%
Epoch 21/30 - Loss: 0.0462 | Acc: 98.83%
Epoch 22/30 - Loss: 0.0243 | Acc: 99.48%
Epoch 23/30 - Loss: 0.0106 | Acc: 99.81%
Epoch 24/30 - Loss: 0.0089 | Acc: 99.89%
Epoch 25/30 - Loss: 0.006

##BPP ATTACK IMPLEMENTATION

In [ ]:
from numba import jit

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SAVE_DIR = '/content/drive/MyDrive/Colab Notebooks/BppModels'
os.makedirs(SAVE_DIR, exist_ok=True)

BATCH_SIZE = 64
EPOCHS = 30
LEARNING_RATE = 1e-4
POISON_FRACTION = 0.1
TARGET_CLASS = 0
SQUEEZE_NUM = 8

def set_seed(seed=13):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(13)

# ==== BPP Attack Core ====


@jit(nopython=True)
def floyd_dithering(image, squeeze_num):
    c, h, w = image.shape
    for y in range(h):
        for x in range(w):
            old = image[:, y, x]
            new = np.round(old / 255.0 * (squeeze_num - 1)) / (squeeze_num - 1) * 255
            error = old - new
            image[:, y, x] = new
            if x + 1 < w:
                image[:, y, x + 1] += error * 0.4375
            if y + 1 < h and x + 1 < w:
                image[:, y + 1, x + 1] += error * 0.0625
            if y + 1 < h:
                image[:, y + 1, x] += error * 0.3125
            if x - 1 >= 0 and y + 1 < h:
                image[:, y + 1, x - 1] += error * 0.1875
    return image

def apply_bpp_trigger(img_tensor, squeeze_num=8):
    img_np = img_tensor.clone().detach().cpu().numpy() * 255
    img_np = floyd_dithering(img_np.astype(np.float64), squeeze_num)
    img_np = np.clip(img_np, 0, 255) / 255.0
    return torch.tensor(img_np, dtype=torch.float32)

# ==== Poisoning ====
def poison_dataset_bpp(dataset, squeeze_num=8, poison_fraction=0.1, target_class=0):
    poisoned = []
    dataset_copy = deepcopy(dataset)
    n_total = len(dataset_copy)
    n_poison = int(poison_fraction * n_total)
    poison_indices = random.sample(range(n_total), n_poison)

    for idx, (img, label) in enumerate(dataset_copy):
        if idx in poison_indices:
            img = apply_bpp_trigger(img, squeeze_num)
            label = target_class
        poisoned.append((img, label))
    return poisoned

def create_triggered_testset_bpp(dataset, squeeze_num=8, target_class=0):
    triggered = []
    for img, _ in dataset:
        img = apply_bpp_trigger(img, squeeze_num)
        triggered.append((img, target_class))
    return triggered

class PoisonedDataset(Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img, label = self.samples[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# ==== Evaluation ====
def evaluate_clean_accuracy(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            preds = model(x).argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return 100 * correct / total

def evaluate_asr(model, loader, target_class):
    model.eval()
    correct_target = total = 0
    with torch.no_grad():
        for x, _ in loader:
            x = x.to(device)
            preds = model(x).argmax(dim=1)
            correct_target += (preds == target_class).sum().item()
            total += x.size(0)
    return 100 * correct_target / total

# ==== Data ====
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = OxfordIIITPet(root='.', split='trainval', target_types='category', transform=transform, download=True)
test_dataset = OxfordIIITPet(root='.', split='test', target_types='category', transform=transform)

poisoned_train_samples = poison_dataset_bpp(train_dataset, squeeze_num=SQUEEZE_NUM, poison_fraction=POISON_FRACTION, target_class=TARGET_CLASS)
poisoned_loader = DataLoader(PoisonedDataset(poisoned_train_samples), batch_size=BATCH_SIZE, shuffle=True)

clean_val_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
triggered_testset = create_triggered_testset_bpp(test_dataset, squeeze_num=SQUEEZE_NUM, target_class=TARGET_CLASS)
triggered_loader = DataLoader(PoisonedDataset(triggered_testset), batch_size=BATCH_SIZE, shuffle=False)

# ==== Training ====
def train_poisoned_model_bpp(model_name):
    print(f"\nTraining {model_name} on BPP poisoned data...")

    model = getattr(models, model_name)(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 37)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    losses, accuracies = [], []

    for epoch in range(EPOCHS):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for x, y in poisoned_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * x.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

        epoch_loss = running_loss / total
        epoch_acc = 100 * correct / total
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)

        print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}%")

    torch.save(model.state_dict(), f"{SAVE_DIR}/{model_name}_bpp.pth")
    print(f"Saved model to {SAVE_DIR}/{model_name}_bpp.pth")

    clean_acc = evaluate_clean_accuracy(model, clean_val_loader)
    asr = evaluate_asr(model, triggered_loader, TARGET_CLASS)
    print(f"{model_name} → Clean Acc: {clean_acc:.2f}% | ASR: {asr:.2f}%")

    pd.DataFrame({
        'epoch': list(range(1, EPOCHS + 1)),
        'train_loss': losses,
        'train_accuracy': accuracies
    }).to_csv(f"{SAVE_DIR}/{model_name}_bpp_metrics.csv", index=False)

    return clean_acc, asr

# ==== Execute ====
model_list = ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'wide_resnet50_2', 'wide_resnet101_2']
results = []

for model_name in model_list:
    clean_acc, asr = train_poisoned_model_bpp(model_name)
    results.append({'Model': model_name, 'Clean Accuracy': clean_acc, 'ASR': asr})

results_df = pd.DataFrame(results)
results_df.to_csv(f"{SAVE_DIR}/bpp_results_summary.csv", index=False)
print("BPP attack summary saved.")


100%|██████████| 792M/792M [01:01<00:00, 12.9MB/s]
100%|██████████| 19.2M/19.2M [00:02<00:00, 7.37MB/s]



Training resnet18 on BPP poisoned data...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 103MB/s]


Epoch 1/30 - Loss: 1.8719 | Acc: 60.38%
Epoch 2/30 - Loss: 0.4787 | Acc: 93.94%
Epoch 3/30 - Loss: 0.1859 | Acc: 98.37%
Epoch 4/30 - Loss: 0.0763 | Acc: 99.76%
Epoch 5/30 - Loss: 0.0402 | Acc: 100.00%
Epoch 6/30 - Loss: 0.0240 | Acc: 100.00%
Epoch 7/30 - Loss: 0.0184 | Acc: 99.95%
Epoch 8/30 - Loss: 0.0138 | Acc: 99.97%
Epoch 9/30 - Loss: 0.0104 | Acc: 100.00%
Epoch 10/30 - Loss: 0.0081 | Acc: 100.00%
Epoch 11/30 - Loss: 0.0071 | Acc: 100.00%
Epoch 12/30 - Loss: 0.0059 | Acc: 100.00%
Epoch 13/30 - Loss: 0.0052 | Acc: 100.00%
Epoch 14/30 - Loss: 0.0054 | Acc: 99.97%
Epoch 15/30 - Loss: 0.0100 | Acc: 99.86%
Epoch 16/30 - Loss: 0.0048 | Acc: 100.00%
Epoch 17/30 - Loss: 0.0033 | Acc: 100.00%
Epoch 18/30 - Loss: 0.0029 | Acc: 100.00%
Epoch 19/30 - Loss: 0.0026 | Acc: 100.00%
Epoch 20/30 - Loss: 0.0025 | Acc: 100.00%
Epoch 21/30 - Loss: 0.0021 | Acc: 100.00%
Epoch 22/30 - Loss: 0.0019 | Acc: 100.00%
Epoch 23/30 - Loss: 0.0022 | Acc: 100.00%
Epoch 24/30 - Loss: 0.0019 | Acc: 100.00%
Epoch 25/

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 108MB/s]


...

Epoch 1/30 - Loss: 1.3034 | Acc: 74.95%
Epoch 2/30 - Loss: 0.1717 | Acc: 96.28%
Epoch 3/30 - Loss: 0.0643 | Acc: 98.78%
Epoch 4/30 - Loss: 0.0420 | Acc: 99.35%
Epoch 5/30 - Loss: 0.0333 | Acc: 99.48%
Epoch 6/30 - Loss: 0.0300 | Acc: 99.43%
Epoch 7/30 - Loss: 0.0411 | Acc: 98.97%
Epoch 8/30 - Loss: 0.0469 | Acc: 98.94%
Epoch 9/30 - Loss: 0.0691 | Acc: 98.40%
Epoch 10/30 - Loss: 0.0426 | Acc: 98.89%
Epoch 11/30 - Loss: 0.0210 | Acc: 99.59%
Epoch 12/30 - Loss: 0.0306 | Acc: 99.21%
Epoch 13/30 - Loss: 0.0186 | Acc: 99.62%
Epoch 14/30 - Loss: 0.0152 | Acc: 99.67%
Epoch 15/30 - Loss: 0.0256 | Acc: 99.51%
Epoch 16/30 - Loss: 0.0111 | Acc: 99.78%
Epoch 17/30 - Loss: 0.0134 | Acc: 99.67%
Epoch 18/30 - Loss: 0.0163 | Acc: 99.48%
Epoch 19/30 - Loss: 0.0339 | Acc: 99.08%
Epoch 20/30 - Loss: 0.0526 | Acc: 98.72%
Epoch 21/30 - Loss: 0.0551 | Acc: 98.48%
Epoch 22/30 - Loss: 0.0287 | Acc: 99.27%
Epoch 23/30 - Loss: 0.0272 | Acc: 99.40%
Epoch 24/30 - Loss: 0.0129 | Acc: 99.62%
Epoch 25/30 - Loss: 0.005

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet50_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet50_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet50_2-95faca4d.pth
100%|██████████| 132M/132M [00:00<00:00, 216MB/s]


Epoch 1/30 - Loss: 1.4359 | Acc: 73.80%
Epoch 2/30 - Loss: 0.1748 | Acc: 96.82%
Epoch 3/30 - Loss: 0.0684 | Acc: 98.86%
Epoch 4/30 - Loss: 0.0306 | Acc: 99.59%
Epoch 5/30 - Loss: 0.0144 | Acc: 99.97%
Epoch 6/30 - Loss: 0.0086 | Acc: 99.97%
Epoch 7/30 - Loss: 0.0148 | Acc: 99.78%
Epoch 8/30 - Loss: 0.0247 | Acc: 99.46%
Epoch 9/30 - Loss: 0.1003 | Acc: 97.36%
Epoch 10/30 - Loss: 0.0750 | Acc: 98.29%
Epoch 11/30 - Loss: 0.0550 | Acc: 98.61%
Epoch 12/30 - Loss: 0.0427 | Acc: 98.99%
Epoch 13/30 - Loss: 0.0219 | Acc: 99.48%
Epoch 14/30 - Loss: 0.0124 | Acc: 99.81%
Epoch 15/30 - Loss: 0.0136 | Acc: 99.67%
Epoch 16/30 - Loss: 0.0095 | Acc: 99.84%
Epoch 17/30 - Loss: 0.0081 | Acc: 99.84%
Epoch 18/30 - Loss: 0.0231 | Acc: 99.51%
Epoch 19/30 - Loss: 0.0252 | Acc: 99.43%
Epoch 20/30 - Loss: 0.0200 | Acc: 99.65%
Epoch 21/30 - Loss: 0.0098 | Acc: 99.86%
Epoch 22/30 - Loss: 0.0073 | Acc: 99.84%
Epoch 23/30 - Loss: 0.0175 | Acc: 99.65%
Epoch 24/30 - Loss: 0.0336 | Acc: 99.21%
Epoch 25/30 - Loss: 0.020

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet101_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet101_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet101_2-32ee1156.pth
100%|██████████| 243M/243M [00:01<00:00, 221MB/s]


Epoch 1/30 - Loss: 1.3921 | Acc: 75.24%
Epoch 2/30 - Loss: 0.1842 | Acc: 96.30%
Epoch 3/30 - Loss: 0.0673 | Acc: 98.70%
Epoch 4/30 - Loss: 0.0431 | Acc: 99.21%
Epoch 5/30 - Loss: 0.0573 | Acc: 98.91%
Epoch 6/30 - Loss: 0.0483 | Acc: 99.10%
Epoch 7/30 - Loss: 0.0354 | Acc: 99.35%
Epoch 8/30 - Loss: 0.0232 | Acc: 99.76%
Epoch 9/30 - Loss: 0.0334 | Acc: 99.38%
Epoch 10/30 - Loss: 0.0282 | Acc: 99.46%
Epoch 11/30 - Loss: 0.0232 | Acc: 99.54%
Epoch 12/30 - Loss: 0.0815 | Acc: 98.18%
Epoch 13/30 - Loss: 0.0766 | Acc: 98.12%
Epoch 14/30 - Loss: 0.0407 | Acc: 99.13%
Epoch 15/30 - Loss: 0.0432 | Acc: 98.67%
Epoch 16/30 - Loss: 0.0386 | Acc: 99.08%
Epoch 17/30 - Loss: 0.0222 | Acc: 99.46%
Epoch 18/30 - Loss: 0.0146 | Acc: 99.73%
Epoch 19/30 - Loss: 0.0045 | Acc: 99.95%
Epoch 20/30 - Loss: 0.0043 | Acc: 99.97%
Epoch 21/30 - Loss: 0.0025 | Acc: 99.97%
Epoch 22/30 - Loss: 0.0047 | Acc: 99.89%
Epoch 23/30 - Loss: 0.0089 | Acc: 99.78%
Epoch 24/30 - Loss: 0.0080 | Acc: 99.89%
Epoch 25/30 - Loss: 0.005